# Modellierung von verteilten Systemen

In [1]:
; internal setup
(do (require '[clojupyter.misc.helper :as helper])
    (require '[cemerick.pomegranate :as pom])
    (helper/add-dependencies '[org.clojure/test.check "0.9.0"])
    (helper/add-dependencies '[org.clojure/math.combinatorics "0.1.4"])
    (helper/add-dependencies '[progrock "0.1.2"])
    (helper/add-dependencies '[me.lomin/sayang "0.3.0"])
    (pom/add-classpath "src")
    (require '[me.lomin.piggybank.accounting.doc :as a-doc]))

nil

Durch die Trends Microservices und Serverless sowie die häufigere Verwendung von Datenbanken, die aufgrund bestimmter Performance-Anforderungen nicht mehr über die traditionellen Konsistenzgarantien verfügen, werden sich Entwicklerteams immer häufiger mit den Problemen von verteilten Systemen auseinander setzen müssen. Zu diesen Problemen gehört auch, wie man Fehler über Servicegrenzen hinweg debuggen kann, oder wie es ermöglicht werden kann, dass die verschiedenen Services unabhängig voneinander deployt werden können. Dieser Artikel jedoch befasst sich mit zwei speziellen Fehlerkategorien, welche im besonderen Maß durch verteilte Systeme provozierten werden: Konsistenzfehler und Nebenläufigkeitsfehler.

Diese Arten von Fehlern sind besonders schwierig zu reproduzieren, können unbemerkt zu Datenkorruption führen und sind mit den etablierten Testvorgehen kaum zu identifiziert.

In der agilen Softwareentwicklung wird ein hohes Maß an technischer Qualität des Softwareinkrements gefordert. Dazu gehört insbesondere, dass die Software automatisiert auf der Ebene des Source-Codes (Unit Tests), im Verbund von Komponenten (Integration Tests) und aus der Perspektive des Kunden (Acceptance Tests) getestet wird. Der Umfang der jeweiligen Testkategorie wird anhand der Testpyramide verdeutlicht:

![Test Pyramide](https://www.borisgloger.com/wp-content/uploads/blog/Testp-300x281-300x281.jpeg "Test Pyramide")

Es wurde die Form einer Pyramide gewählt wurde, weil der Großteil der Tests eines Systems aus Unit Tests bestehen sollte. Entsprechen die Unit Tests den FIRST-Kriterien, so liefern Unit Tests am schnellsten und zuverlässigsten Feedback. Zwar liefern Integrationsstests und besonders Akzeptanztests am langsamsten Feedback, dafür 

Zwar liefern Akzeptanztests am langsamsten Feedback und ihre Pflege gestaltet sich am aufwändigsten, dafür sind die Tests am nächsten zur Realität. Ein Akzeptanztests greift auf die gleichen Testmöglichkeiten wir ein menschlicher Tester oder ein Benutzer des Systems zurück.

Integrationsstests stellen einen Mittelweg zwischen Unit Tests und Akzeptanztests dar. Mit Integrationsstest testet man das technische Zusammenspiel von Komponenten oder Services, jedoch nicht über die Benutzeroberfläche des Systems, sondern ähnlich wie bei Unit Tests in einem künstlich erzeugten Test-Setup. Das Setup ist weniger künstlich als bei Unit Tests, jedoch nicht so realitätsnah wie bei Akzeptanztest.

Der Begriff Testpyramide spiegelt den breiten Konsens innerhalb der agilen Entwicklungscommunity wieder, dass der beste Kompromiss zwischen Aufwand und Ertrag von Tests zu einer Verteilung von Unit Tests zu Integrationstests und Akzeptanztests zu der Form einer Pyramide führt.

Damit Unit Tests schnell und zuverlässig Feedback liefern, sollten sie den FIRST-Kriterien entsprechen:

* Fast: Unit Tests sollen schnell sein
* Isolated: Unit Tests sollten unabhängig von externen Abhängigkeiten wie einer Datenbank sein
* Repeatable: Unit Tests sollen wiederholbar, also deterministisch sein
* Self Validating: Am Ende eines Unit Tests sollte dieser ein eindeutiges Ergebnis zurückliefern
* Timely: Man sollte Unit Tests möglichst früh schreiben, am besten bereits vor dem Produktivcode

Durch die FIRST-Kriterien wird klar, dass Unit Tests sowohl für Konsistenzfehler als auch für Nebenläufigkeitsfehler das falsche Werkzeug sind. Angenommen, man möchte ein System, das eine Datenbank mit dem Konsistenzmodell “Eventual Consistency” nutzt, auf Konsistenzfehler testen. Da ein Unit Test, aufgrund des Isolated-Kriteriums, keine Abhängigkeit zu einer Datenbank haben darf, kann das System nicht in Zusammenspiel mit der Datenbank in einem Unit Test geprüft werden. Der Workaround, stattdessen eine einer Attrappe der Datenbank zu verwenden, würde nur zu einem Gefühl der falschen Sicherheit verleiten, da das Verhalten der Datenbank in vielerlei Hinsicht nicht exakt nachgestellt werden kann. Auch zum Testen von Nebenläufigkeit sind Unit Tests eher ungeeignet, treten Nebenläufigkeitsprobleme doch vor allem zufällig und indeterministisch auf. Genau diese Eigenschaften schließt das Repeatable-Kriterium aber aus.

Sowohl für Integrationstests als auch noch stärker für Akzeptanztest gilt, dass auf dieser Ebene die Kombination der möglichen Setups, Schritte und Konstellationen zu so einer hohen Zahl an Testkombinationen führt, dass die Wahrscheinlichkeit gering ist, alle Konsistenz- und Nebenläufigkeitsfehler zu entdecken.

## Modellierung als Alternative zur Testpyramide

Eine Alternative zum Testen mit den Mitteln der Testpyramide stellt die Modellierung des verteilten oder nebenläufigen Systems dar. Das bedeutet, man abstrahiert von dem konkreten Code und konzentriert sich auf das Verhalten und die Eigenschaften des Systems. Dazu stellt man folgende Fragen:
Welche Events können auftreten?
Wie ist der Zustand des Systems nach einem bestimmten Event?
Welche Bedingung muss das System schlussendlich und welche Bedingungen jederzeit erfüllen?
Die Antworten auf diese Fragen bilden ein Modell des zu testenden Systems. Ein Modellprüfer kann damit jegliche Kombination von Events generieren und automatisiert testen, ob nach jedem Event die erforderlichen Bedingungen an den Zustand des Systems erfüllt sind. Findet der Modellprüfer eine Verletzung einer Bedingung, so hat man einen logischen Fehler im Design des Systems gefunden. Findet der Modellprüfer keinen Fehler, so erfüllt das Design des Systems die formulierten Bedingungen und die Übersetzung des Designs in die gewünschte Programmiersprache und Laufzeitumgebung kann beginnen. Da bei der Implementierung des Designs Übersetzungsfehler passieren könnten, die das geprüfte Modell verletzen würden, kann man auf die Werkzeuge der Testpyramide weiterhin nicht verzichten.

Um den Nutzen und die konkrete Anwendung von Modellierung im Zusammenspiel mit einem Modellprüfer zu demonstrieren, werden im folgenden zwei Beispiele behandelt. Das erste Beispiel überprüft ein verteiltes System auf Konsistenzverletzungen, das zweite Beispiel zeigt auf, wie man Nebenläufigkeitsfehler identifizieren und ausschließen kann.

## Beispiel 1: Konsistenz im verteilten System

Als Beispiel für ein verteiltes Systems soll ein digitales Sparschwein mit Buchhalterfunktion für Familien dienen. Um das Beispiel möglichst einfach zu halten, muss das Sparschwein ausschließliche die Einzahlung und die Abhebung von 1-Euro-Münzen unterstützen. Jede Einzahlung und Abhebung wird als Event in einem Dokument einer Dokumentendatenbank festgehalten. Dadurch können alle Ein- und Auszahlungen des Sparschweins nachvollzogen werden. Es darf nicht möglich sein, dass ein Nutzer einen Euro abhebt, obwohl keine Münzen mehr im Sparschwein sind. Die Prüfung dieser Bedingung soll über eine andere Datenbank erfolgen, welche nur den aktuellen Betrag innerhalb des Sparschweins festhält.

Wie eingangs erwähnt, sind folgende Fragen zu beantworten:

1. Welche Events können auftreten?
1. Wie ist der Zustand des Systems nach einem bestimmten Event?
1. Welche Bedingung muss das System schlussendlich und welche Bedingungen jederzeit erfüllen?

Im folgendem starten wir den Modellierungsprozess und finden Antworten auf diese 3 Fragen.

### Welche Events können auftreten?

Um die erste Frage zu beantworten, sind wir bereits gezwungen, das System auf einer hohen Abstraktionsebene in Schritte zu modellieren. Dadurch erkennen wir auch, welche Schritte atomar sind und welche nicht. Beispielsweise können wir nicht gleichzeitig aus einer Dokumentendatenbank lesen und gleichzeitig in diese schreiben. Würde man allein diesen Modellierungsschritt gehen, ließen sich bereits dadurch viele Fehler im Design eines verteilten Systems vermeiden.

In unserem vorliegenden Fall kann man die Events aus den Schritten ableiten, die ein mögliches Buchhaltersparschweinsystem durchführen würde.



Für jede Transaktionsanfrage eines Nutzers werden diese Schritte ausgeführt. Das System hat keinen Einfluss zu welchem Zeitpunkt Transaktionsanfragen an das System gestellt werden. Das System bestimmt aber, welche Schritte danach ausgeführt werden, in welcher Reihenfolge und ob dies synchron oder asynchron geschieht. Diese Festlegungen bezeichnen wir als "Event-Modell”.

* Der Inhalt des Sparschweins wurde ausgelesen
* Familienmitglieder haben die Anfrage zur Einzahlung eines Euros gestellt
* Familienmitglieder haben die Anfrage zur Abhebung eines Euros gestellt
* Aus der Dokumentendatenbank wurde das aktuelle Dokument ausgelesen
* Eine Transaktion (Abhebung oder Einzahlung) wurde festgehalten
* Der Betrag des Sparschweins wurde aktualisiert

Wir modellieren die Events absichtlich als Ereignisse, die erfolgreich stattgefunden haben. Später wird klarer werden, warum wir die Events nicht als Anweisungen wie z.B. “Halte Transaktion fest” modellieren.

Um diese Events als Datenstrukturen zu modellieren, nutze ich die Programmiersprache Clojure. Diese Sprache verfügt über nur wenig Syntax und man kann Datenstrukturen kurz und prägnant darstellen. Funktionsaufrufe in Clojure folgen der Prefix-Notation. Statt

```
f (x, y)
```

schreibt man

```clojure
(f x y)
```

ein Vektor wird so dargestellt:

In [2]:
[1 2 3]

[1 2 3]

und eine Set (eine Menge von Elementen) so:

In [3]:
#{1 2 3}

#{1 3 2}

Dokumente, in anderen Sprachen auch als Maps, Dicts, oder Records bezeichnet, lassen sich wie folgt darstellen:

In [4]:
{"key0" "value0" "key1" "value1"}

{"key0" "value0", "key1" "value1"}

Keys sind nicht auf Strings beschränkt. Ein Vektor, eine Menge oder eine Map können auch Keys oder Values einer Map sein:

In [5]:
{["key" 5] {"inner-key" "value"}}

{["key" 5] {"inner-key" "value"}}

Häufig werden Clojure Keywords als Schlüssel verwendet, da man mit Keywords sehr schnell Gleichheit feststellen kann und man einfach auf die Werte einer Map zugreifen kann. Keyword beginnen immer mit einem Doppelpunkt:

In [6]:
:key-0

:key-0

In [7]:
(= :key-0 :key-1)

false

In [8]:
(= :key-1 :key-1)

true

In [9]:
(:key {:key "value"})

"value"

Die oben Events auf dem Event-Modell werden dementsprechend wie folgt als Datenstrukturen modelliert:

In [10]:
(a-doc/print-accounting-events)

(defn example-accounting-events []
  [:process
   {:amount 1, :process-id 0}
   :doc
   "Eine Anfrage mit der Prozess-Id 0 zur Einzahlung von 1 Euro wurde gestartet."]
  [:process
   {:amount -1, :process-id 1}
   :doc
   "Eine Anfrage mit der Prozess-Id 1 zur Abhebung von 1 Euro wurde gestartet."]
  [:accounting-read
   {:amount 1, :process-id 0}
   :doc
   "Die notwendigen Daten aus dem Buchhaltungssystem wurden abgefragt. Das Ergebnis der Abfrage wurde abgelegt. Ausgelöst wurde dieses Event durch den Prozess mit der Prozess-Id 0."]
  [:accounting-write
   {:amount 1, :process-id 0}
   :doc
   "Die Transaktion wurde im Buchhaltungssystem festgehalten."]
  [:balance-write
   {:amount 1, :process-id 0}
   :doc
   "Das Saldo des Sparschweins wurde aktualisiert."])


"listing-1"

### Wie ist der Zustand des Systems nach einem bestimmten Event?

Das Buchhaltungssystem speichert die Einzahlungen und Abhebung in einem Dokument einer Dokumentendatenbank. Auch das Buchhaltungssystems kann man als Datenstruktur modellieren. Ein Beispiel:

In [11]:
(a-doc/accounting-subsystem)

{:accounting {:transfers {0 1, 1 -1}}}


"listing-2"

Dieser Datenstruktur kann man entnehmen, dass der Prozess mit der ID 0 eine Einzahlung von einem Euro und der Prozess mit der ID 1 eine Abhebung von einem Euro zur Folge hatte.

Den aktuellen Eurowert des Sparschweins nennen wir "Balance” und dieser soll in einer eigenen Datenbank festgehalten werden. Da es sich nur um einen Wert handelt, ist es egal, ob wir von einer Dokumentendatenbank, einem Key-Value-Store oder einer relationalen Datenbank ausgehen. Wir modellieren den Zustand der Datenbank folgendermaßen:

In [12]:
(a-doc/balance-subsystem)

{:balance {:amount 0}}


"listing-3"

Wir vereinen das Modell der Buchhaltung und das Modell des aktuellen Inhalts des Sparschweins in einer Datenstruktur, die wir Universum nennen:

In [13]:
(a-doc/universe-0)

{:accounting {:transfers {0 1, 1 -1}}, :balance {:amount 0}}


"listing-4"

Die Events, die wir anhand der ersten Modellierungsfrage identifiziert haben, verändern das Universum. Wenn beispielsweise im oben beschriebenen Beispieluniversum das Event

In [14]:
(a-doc/event-0)

[:accounting-write {:amount 1, :process-id 5}]


"listing-5"

stattgefunden hat, verändert sich das Universum dementsprechend wie folgt:

In [15]:
(a-doc/universe-after-update)

{:accounting {:transfers {0 1, 1 -1, 5 1}}, :balance {:amount 1}}


"listing-6"

### Welche Bedingung muss das System schlussendlich und welche Bedingungen jederzeit erfüllen?

Eine Bedingungen haben wir bereits angesprochen: Man kann nicht mehr Münzen aus dem Sparschwein nehmen, als tatsächlich vorhanden sind

Es gibt aber noch weitere:

* Die Münztransaktionen werden in einem Dokument einer Dokumentendatenbank festgehalten. Diese Datenbank bietet Datenbanktransaktionssicherheit auf Dokumentenebene an. Das bedeutet, man kann keine zwei Dokumente gleichzeitig verändern. Versucht man es trotzdem, kann es passieren, dass ein Dokument verändert wird, beim Ändern des zweiten aber ein Fehler auftritt und die Änderung nicht übernommen wird. Die Änderung des ersten Dokumentes kann nicht automatisiert durch die Datenbank rückgängig gemacht werden. Für die Dokumentendatenbank waren das zwei unterschiedliche Transaktionen.
* Wenn eine Münztransaktionen einmal in ein Buchhaltungsdokument eingetragen wurde, darf diese Münztransaktion nicht mehr aus dem Buchhaltungsdokument entfernt werden

## Automatisierte Schwachstellen im Design mithilfe eines Modellcheckers finden

Auch wenn allein durch den Modellierungsprozess Klarheit über Fehlerquellen geschaffen wurden, so wollen wir noch einen Schritt weitergehen und unser Event-Modell automatisiert prüfen. Dazu müssen wir uns erst einmal eine Vorstellung machen, wie wir Zeit und insbesondere Gleichzeitigkeit modellieren.

Die erste Idee ist es, die Zeit als aufeinander folgende atomare Zeitfenster vorzustellen. In einem Zeitfenster hat genau ein Event stattgefunden. Die Folge an aufeinanderfolgenden Zeitfenstern bezeichnen wir als Zeitstrahl. Hier ein Beispiel für einen konkreten Zeitstrahl

In [16]:
(a-doc/example-timeline-0)

[[:process {:amount -1, :process-id 0}]
 [:process {:amount 1, :process-id 1}]
 [:process {:amount -1, :process-id 2}]]


"listing-7"

Dieser Zeitstrahl besteht aus drei Zeitfenstern mit drei unterschiedlichen Events des gleichen Typs.

Ein häufiges Problem bei verteilten Systemen ist die Nichtberücksichtigung von gleichzeitig auftretenden Vorgängen. Es stellt sich also die Frage, wie man mit der oben formulierten Vorstellung von Zeit gleichzeitig auftretende Ereignisse modelliert. Dazu schauen wir uns einen anderen Zeitstrahl an:

In [17]:
(a-doc/example-timeline-1)

[[:process {:amount 1, :process-id 0}]
 [:process {:amount -1, :process-id 1}]
 [:accounting-write {:amount 1, :process-id 0}]
 [:accounting-write {:amount -1, :process-id 1}]]


"listing-8"

Dieser Zeitstrahl sagt aus, dass zuerst die Münztransaktion mit der ID 0 gespeichert wird und dann die Münztransaktion mit der ID 1. Wie kann man aber modellieren, dass beide Münztransaktion gleichzeitig gespeichert werden? Zur Hälfte haben wir das bereits. Der Zeitstrahl in Listing X sagt sowohl aus, dass die Münztransaktion 1 nach der Münztransaktion 0 erfolgt ist, also auch, dass beide Münztransaktionen gleichzeitig erfolgt sind und sich die beiden schreibenden Zugriffe gegenseitig überschrieben haben, mit Münztransaktion 1 als sichtbaren Gewinner dieses konkurrierenden Zugriffes. Wenn aber die Münztransaktionen 0 und 1 exakt gleichzeitig erfolgt sind, hätte es aber genauso gut sein können, dass Münztransaktion 0 der sichtbare Gewinner dieses konkurrierenden Zugriffes hätte sein können. Durch Gleichzeitigkeit zweier Ereignisse entstehen also zwei gleichwertige Paralleluniversen. Da ein Universum durch die aufeinanderfolgende Anwendung der Ereignisse eines Zeitstrahls entsteht, müssen wir garantieren, dass wir neben dem Zeitstrahl aus listing-example-timeline-1 einen weiteren Zeitstrahl betrachten, der folgendermaßen aussieht:

In [18]:
(a-doc/example-timeline-2)

[[:process {:amount 1, :process-id 0}]
 [:process {:amount -1, :process-id 1}]
 [:accounting-write {:amount -1, :process-id 1}]
 [:accounting-write {:amount 1, :process-id 0}]]


"listing-9"

Ein Konzept fehlt noch. Wir benötigen noch ein Nicht-Event. Dieses ist notwendig, um zum Beispiel modellieren zu können, dass ein Datenbankzugriff verzögert wird oder fehlschlägt. Solch ein Event wird als "Stuttering" bezeichnet.

Der folgende Zeitstrahl beschreibt beispielsweise ein Szenario, in dem die Buchhaltunsdatenbank nicht erreichbar ist:

In [19]:
(a-doc/accounting-db-down-timeline)

[[:process {:amount -1, :process-id 0}] [:stuttering] [:stuttering]]


"listing-10"

Nun haben wir alle Voraussetzungen erfüllt, um unsere Event-Modelle automatisiert prüfen zu lassen.

## Das erste Event-Modell: Nebenläufig

Das erste Event-Modell beschreibt die erste Idee einer Architektur unseres verteilten System. Als Datenbank für die Buchhaltung ist eine Dokumentendatenbank gesetzt. Als Datenbank für den Saldo entscheiden wir uns für eine relationale Datenbank. Die Nutzeranfragen werden von mehreren Instanzen des von uns entwickelten Sparschweinservices entgegengenommen, bearbeitet und in die jeweiligen Datenbanken geschrieben:

![Nebenläufige Architektur](images/concurrent_piggybank.jpg "Nebenläufige Architektur")

Das Event-Modell, das diese Architektur beschreibt, formulieren wir als Datenstruktur mithilfe einer Clojure Map:

In [20]:
(a-doc/multi-threaded-simple-model)

(def multi-threaded-simple-model
 (make-model
   {START
    (all
      (generate-incoming
        multi-threaded
        [:process {:amount 1}]
        [:process {:amount -1}])
      (always [:stuttering])),
    :process (all (then :accounting-read)),
    :accounting-read (all (then :accounting-write)),
    :accounting-write (all (then :balance-write)),
    :balance-write (continue)}))


"listing-11"

Dieses Modell sagt aus, dass zu jeder Zeit Nutzeranfragen mit Einzahlungen (```{:euro 1}```) oder Abhebungen (```{:euro -1}```) eintreffen können, sowie dass das Nicht-Event ```[:stuttering]``` jederzeit auftreten kann. Jede Nutzeranfragen wird mit einer eindeutigen ID versehen. Alle Events eines Zeitstrahls mit der gleichen ID bezeichnen wir als "Prozess". Unser Service wird nicht von selbst die Buchhaltung anfragen; das wird erst durch das Event ```:transaktion-gestartet``` ausgelöst. Das wird durch den Funktionsaufruf ```(then :accounting-read)``` modelliert. Die Funktion ```(all (x) (y) (z))``` sagt aus, dass alles in x, y und z gleichermaßen passieren kann. Im Gegensatz dessen werden wir in späteren Modellen die Funktion ```(choose (x) (y) (z))``` verwenden, die beschreibt, dass nur eins von x, y, oder z passieren kann.

Um zu verdeutlichen, welche Zeitstrahlen aus ein Modell resultieren, folgend ein vereinfachtes Beispielmodell:

In [21]:
(a-doc/reduced-multi-threaded-simple-model)

(def reduced-multi-threaded-simple-model
 (make-model
   {START
    (all
      (generate-incoming multi-threaded [:process {:amount 1}])
      (always [:stuttering])),
    :process (all (then :accounting-read)),
    :accounting-read (all (then :accounting-write)),
    :accounting-write (continue)}))


"listing-12"

Dieses Modell würde zu folgenden Zeitstrahlen der Länge 2 führen:

In [22]:
(a-doc/timelines-reduced-multi-threaded-simple-model 2)

#{[[:process {:amount 1, :process-id 0}]
   [:process {:amount 1, :process-id 1}]]
  [[:stuttering] [:stuttering]]
  [[:process {:amount 1, :process-id 0}]
   [:accounting-read {:amount 1, :process-id 0}]]
  [[:stuttering] [:process {:amount 1, :process-id 0}]]
  [[:process {:amount 1, :process-id 0}] [:stuttering]]}


"listing-13"

Ein möglicher Zeitstrahl der Länge 4 für das vereinfachte Event-Modell ist

In [23]:
(a-doc/valid-sample-timeline-reduced-multi-threaded-simple-model 4)

[[:process {:amount 1, :process-id 0}]
 [:stuttering]
 [:stuttering]
 [:stuttering]]


"listing-14"

Der Zeitstrahl

In [24]:
(a-doc/print-invalid-sample-timeline-reduced-multi-threaded-simple-model)

(defn invalid-sample-timeline-reduced-multi-threaded-simple-model []
  [[:process {:amount 1, :process-id 0}]
   [:process {:amount 1, :process-id 1}]
   [:accounting-read {:amount 1, :process-id 0}]
   [:accounting-read {:amount 1, :process-id 0}]])


"listing-15"

ist jedoch mit dem formulierten Event-Modell nicht möglich. Nach dem Event ```[:accounting-read {:amount 1, :process-id 0}]``` kann es kein ```:accounting-read``` Event mit der ID 0 mehr geben.

Nachdem wir das Event-Modell unseres nebenläufigen, zustandslosen Sparschweinservice beschrieben haben, können wir die Bedingungen automatisiert prüfen, die wir in Abschnitt Bedingungen formuliert haben. Dazu lassen wir alle möglichen Zeitstrahlen generieren, interpretieren die Events entlang des Zeitstrahls und erzeugen für jedes Zeitfenster ein neues Universum, für welches wir alle formulierten Bedingungen prüfen. Zusätzlich bauen wir noch in den Interpreter ein, dass das Event ```:accounting-read``` auch den Abschluss der Saldo-Prüfung bedeutet. War die Saldo-Prüfung negativ, werden alle folgenden Events des entsprechenden Prozess ignoriert. Dadurch können wir die Trennung zwischen allen möglichen Events, welche nur Abhängig vom Event-Modell sind, und der Interpretation der Events, welche abhängig vom Zustand des Universums ist, aufrecht erhalten.

Der Model-Checker kommt zu folgendem Ergebnis:

In [36]:
(a-doc/check-multi-threaded-simple-model)


{:check-count 64938,
 :max-check-count 311171,
 :property-violated
 {:name :there-must-be-no-lost-updates,
  :timeline
  [[:process {:amount 1, :process-id 0}]
   [:process {:amount 1, :process-id 1}]
   [:accounting-read {:amount 1, :process-id 0}]
   [:accounting-read {:amount 1, :process-id 1}]
   [:accounting-write {:amount 1, :process-id 0}]
   [:balance-write {:amount 1, :process-id 0}]
   [:accounting-write {:amount 1, :process-id 1}]]},
 :accounting {:transfers {1 1}},
 :balance {:amount 1, :processes #{0}}}


"listing-24"

Insgesamt gibt es 44.453 mögliche Zeitstrahlen der Länge 7, was zu 311.171 zu prüfenden Zeitfenstern führt. Nach

In [26]:
(a-doc/number-of-checked-time-slots)

 62234/311171    19% [=========                                         ]  ETA: 00:07

64938

geprüften Zeitfenstern hat der Model-Checker einen Zeitstrahl gefunden, der zu einem Universum führt, welches mindestens eine Bedingung verletzt.

Der gefundene Zeitstrahl führt dazu, dass zwei Prozesse aus der Buchhaltungsdatenbank das gleiche Dokument lesen, dieses um ihre Münztransaktion ergänzen und zurückschreiben. Dabei überschreibt der Prozess 1 das Dokument, welches der Prozess 0 in die Buchhaltungsdatenbank geschrieben hat.

Um diese Verletzung erkennen zu können, fügen wir zu jeder Münztransaktion auch noch die Menge an IDs an, die zu dieser Transaktion geführt haben. Außerdem halten wir im Saldo-Modell die Menge an IDs fest, die zu einer Änderung des Saldos geführt haben. Diese Daten, die den Modellen nur hinzugefügt werden, damit man anhand dieser Verletzungen der Eigenschaften feststellen kann, sonst aber für das Modell irrelevant sind, bezeichnet man auch als “Ghost Data”. Wenn die Menge aller IDs aus den Büchern keine Supermenge der Menge der IDs aus dem Saldo-Modell ist, haben wir sogenannte “Lost Updates” vorliegen. Es ist ok, wenn das Saldo-Modell noch nicht alle Updates aus den Büchern widerspiegelt, aber im umgekehrten Fall ist es dadurch erwiesen, dass wir Updates aus den Büchern verloren haben.

In [27]:
(a-doc/lost-updates?)

(defn lost-updates? [universe]
  (not
    (set/superset?
      (all-event-ids-from-accounting universe)
      (all-event-ids-from-balance universe))))


"listing-17"

## Das zweite Event-Modell: Single Writer

Um dem Problem der Lost-Updates zu begegnen, gibt es verschiedene Möglichkeiten, die sich erheblich in ihrer Komplexität unterscheiden. Die simpelste Variante ist es, nur noch eine Instanz des Sparschweinservices mit nur einem schreibenden Thread zu betreiben. Dies wird auch als “Single Writer”-Pattern bezeichnet.

In Listing 7 haben wir ein vereinfachtes Event-Modell beschrieben und Listing 8 zeigt alle  Zeitstrahlen der Länge 3, welche daraus folgen. Das gleiche Modell im Single Writer Mode würde so aussehen:

In [28]:
(a-doc/reduced-single-threaded-simple-model)

(def reduced-single-threaded-simple-model
 (make-model
   {START
    (all
      (generate-incoming single-threaded [:process {:amount 1}])
      (always [:stuttering])),
    :process (all (then :accounting-read)),
    :accounting-read (all (then :accounting-write)),
    :accounting-write (continue)}))


"listing-18"

Dieses Modell würde zu folgenden Zeitstrahlen der Länge 3 führen:

In [37]:
(a-doc/timelines-reduced-single-threaded-simple-model 3)

#{[[:process {:amount 1, :process-id 0}]
   [:accounting-read {:amount 1, :process-id 0}]
   [:stuttering]]
  [[:process {:amount 1, :process-id 0}]
   [:stuttering]
   [:accounting-read {:amount 1, :process-id 0}]]
  [[:stuttering] [:stuttering] [:stuttering]]
  [[:stuttering] [:stuttering] [:process {:amount 1, :process-id 0}]]
  [[:stuttering]
   [:process {:amount 1, :process-id 0}]
   [:accounting-read {:amount 1, :process-id 0}]]
  [[:process {:amount 1, :process-id 0}] [:stuttering] [:stuttering]]
  [[:stuttering] [:process {:amount 1, :process-id 0}] [:stuttering]]
  [[:process {:amount 1, :process-id 0}]
   [:accounting-read {:amount 1, :process-id 0}]
   [:accounting-write {:amount 1, :process-id 0}]]}


"listing-25"

Es gibt für dieses vereinfachte Modell nur

In [30]:
(a-doc/count-timelines-reduced-single-threaded-simple-model 3)

8

statt

In [31]:
(a-doc/count-timelines-reduced-multi-threaded-simple-model 3)

15

mögliche Zeitstrahlen. Deshalb prüfen wir das vereinfachte Modell mit einer größeren Zeitstrahllänge, beispielsweise mit 14 statt wie zuvor mit 7.

Das gesamte Event-Modell unterscheidet sich nur in Zeile 1 von unserem ersten Event-Modell:

In [32]:
(a-doc/single-threaded-simple-model)

(def single-threaded-simple-model
 (make-model
   {START
    (all
      (generate-incoming
        single-threaded
        [:process {:amount 1}]
        [:process {:amount -1}])
      (always [:stuttering])),
    :process (all (then :accounting-read)),
    :accounting-read (all (then :accounting-write)),
    :accounting-write (all (then :balance-write)),
    :balance-write (continue)}))


"listing-20"

Der Model-Checker liefert uns folgendes Ergebnis:

In [33]:
(a-doc/check-single-threaded-simple-model)

2190/2191    99% [================================================= ]  ETA: 00:00
{:check-count 2191, :max-check-count 2191}


"listing-21"

Es wurden also keine Fehler gefunden. Wenn wir unser System nach dem Single-Writer-Prinzip entwerfen, würden wir keine unserer Bedingung verletzen. Durch Bugs in der Implementierung des tatsächlichen Systems könnten wir es dennoch schaffen, unsere Bedingungen zu verletzen, aber im Gegensatz zum nebenläufigen Design sind diese Fehler nicht Design-inherent.

## Ein weiteres Modell: Saldo wird In-Memory gehalten und eine eventuell-konsistente Buchhaltungsdatenbank verwendet

Wir wollen nun ein weiteres Design unter die Lupe nehmen. Wir behalten das Single-Writer-Prinzip, aber bei jedem Start der Anwendung werden alle Überweisungen aus der Datenbank gelesen und das daraus resultierende Saldo wird In-Memory von der Single-Writer-Instanz gehalten. Die Überweisungen werden in einer Dokument-Datenbank persistiert, welche nicht mehr garantiert, dass man seine eigenen Writes lesen kann. Die Anwendung reagiert auf dieses Hindernis, indem der Single-Writer nach dem Schreiben in die Datenbank erst so lange die Datenbank abfragt, bis er seinen eigenen Write lesen konnte. Erst danach fährt er mit den nächsten Schritten seines Algorithmuses fort.


Da das modellierte System Stateful ist, müssen wir im Vergleich zu den vorherigen Modellen den Restart des Systems modellieren. Beim Restart kann passieren, dass wir einen Write noch nicht lesen können, der sich aber nach dem abgeschlossenen Restart in der Buchhaltungsdatenbank materialisiert. Wir modellieren das so, dass der Restart einen beliebigen früheren Zustand des Universums zu Gesicht bekommt, und anhand dessen den Saldo neu aufbaut. Das resultierende Modell sieht dementsprechend so aus:

In [34]:
(a-doc/single-threaded+inmemory-balance+eventually-consistent-accounting-model)

(def single-threaded+inmemory-balance+eventually-consistent-accounting-model
 (make-model
   {START
    (all
      (generate-incoming
        single-threaded
        [:process {:amount 1}]
        [:process {:amount -1}])),
    :restart (all (restart)),
    :process
    (choose
      (then-for-every-past-time-slot :restart)
      (then :accounting-read)),
    :accounting-read
    (choose
      (then-for-every-past-time-slot :restart)
      (then :accounting-write)),
    :accounting-write
    (choose
      (then-for-every-past-time-slot :restart)
      (then :balance-write)),
    :balance-write
    (choose (then-for-every-past-time-slot :restart) (restart))}))


"listing-22"

Der Model-Checker findet einen fatalen Bug:

In [38]:
(a-doc/check-single-threaded+inmemory-balance+eventually-consistent-accounting-model)


{:check-count 27131,
 :max-check-count 81120,
 :property-violated
 {:name :accounting-balance-must-always-be>=0,
  :timeline
  [[:process {:amount 1, :process-id 0}]
   [:accounting-read {:amount 1, :process-id 0}]
   [:accounting-write {:amount 1, :process-id 0}]
   [:balance-write {:amount 1, :process-id 0}]
   [:process {:amount -1, :process-id 1}]
   [:accounting-read {:amount -1, :process-id 1}]
   [:accounting-write {:amount -1, :process-id 1}]
   [:balance-write {:amount -1, :process-id 1}]
   [:restart {:past 3}]
   [:process {:amount -1, :process-id 2}]
   [:accounting-read {:amount -1, :process-id 2}]
   [:accounting-write {:amount -1, :process-id 2}]]}}


"listing-26"

Unser Design ermöglicht es, dass man sich mehr Geld aus dem Sparschwein nimmt, als vorhanden ist. Dank dem Model-Checker können wir den Bug fixen, bevor wir mit unserem Sparschwein in Produktion gehen und von unseren Nutzern ausgeraubt werden.